# PM2.5 Prediction Analysis

## Introduction

Air pollution is a critical environmental issue affecting millions of people worldwide, with particularly severe impacts in rapidly developing urban areas. Beijing, one of the world's largest cities and a major economic hub, has been grappling with significant air quality challenges for years. Among the various pollutants, PM2.5 (fine particulate matter with a diameter of 2.5 micrometers or less) is of particular concern due to its severe health impacts.

### The Impact of PM2.5
PM2.5 particles are small enough to penetrate deep into the lungs and even enter the bloodstream, leading to a range of health problems including:

- Respiratory issues (asthma, bronchitis)
- Cardiovascular diseases
- Reduced lung function
- Increased risk of stroke
- Premature death in people with heart or lung disease

Beyond health impacts, high levels of PM2.5 also affect visibility, climate, and can have significant economic consequences due to increased healthcare costs and reduced productivity.

## 1. Import Libraries

In this section, we import all necessary libraries for data manipulation, visualization, machine learning, and time series analysis. These libraries include pandas, numpy, matplotlib, seaborn, scikit-learn, and others specific to time series analysis and advanced machine learning models like XGBoost.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#import linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# import decision tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

import xgboost as xgb
import os

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller, acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats

import joblib
import warnings


## 2. Load and Preprocess Data

Our analysis uses a comprehensive dataset that combines PM2.5 measurements from the U.S. Embassy in Beijing with meteorological data from Beijing Capital International Airport. This hourly dataset provides a rich source of information for understanding the interplay between various environmental factors and PM2.5 concentrations.

### Dataset Features:

1. **Temporal Information**: Year, Month, Day, Hour
2. **Target Variable**: PM2.5 concentration
3. **Meteorological Factors**: Dew Point, Temperature, Pressure, Wind direction and speed
4. **Precipitation Indicators**: Cumulated hours of snow and rain

In this section, we:
1. Load the CSV file containing Beijing PM2.5 data.
2. Define and apply the `preprocess_data` function to clean the data, handle missing values, and create datetime features.
3. Create dataframe `X` for feature-based models.
4. Check for Seasonality and Stationarity

   Here, we perform checks for seasonality and stationarity in the PM2.5 time series:
   1. The `check_seasonality` function performs multiple checks: 
      - Seasonal decomposition
      - Autocorrelation
      - Statistical test for seasonality
   2. The `check_stationarity` function uses the Augmented Dickey-Fuller test to check for stationarity.
   3. We apply both functions to our PM2.5 data to understand its temporal characteristics.
   
5. Data Splitting

   In this step, we split our data:
   1. We create a test set (`X_test_final`) containing data from December 28, 2014 to December 31, 2014. These two are the last days of the dataset and roughly represent the 100 data points that I need to predict with my mode (24*4)
   
   2. The remaining data is kept in `X` for our main analysis and model training.

In [ ]:
# for docker
df = pd.read_csv('beijing_pm_25.csv')
display(df.head())

# for local
#df = pd.read_csv('/Users/andresjr/Documents/capgemini/Task_2_new/data/beijing_pm_25.csv')
#display(df.head())


def preprocess_data(df):

    pd.options.mode.chained_assignment = None
    # Drop rows with missing target values
    print("Shape before dropping missing values:", df.shape)
    df = df.dropna(subset=['pm2.5'])

    #print("Shape before dropping missing values:", df.shape)
    #drop rows with missing values
    df = df.dropna()
    print("Shape after dropping missing values:", df.shape)

        # Combine columns into a datetime object
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

    # Convert to the desired format
    df['date'] = df['datetime'].dt.strftime('%Y-%m-%d %H:00:00').astype('datetime64[ns]')

    # Drop the original columns
    df = df.drop(['year', 'month', 'day', 'hour', 'datetime'], axis=1)

    # Select features
    features = ['DEWP', 'TEMP', 'PRES', 'Iws']

    y = df['pm2.5']
    X = df[features]

    # Add the date column back to the dataframe as the first column do not use insert
    X['date'] = df['date']
    X['pm2.5'] = y
    
    X = X.set_index('date')

    return X

def check_seasonality(y):

    """
    Perform multiple checks for seasonality in the time series.
    """
    
    color_pal = sns.color_palette()
    plt.style.use('fivethirtyeight')
    
    # 1. Visual check with seasonal decomposition
    result = seasonal_decompose(y, model='additive', period=365)
    
    plt.figure(figsize=(12, 10))
    plt.subplot(411)
    plt.plot(result.observed)
    plt.title('Observed')
    plt.subplot(412)
    plt.plot(result.trend)
    plt.title('Trend')
    plt.subplot(413)
    plt.plot(result.seasonal)
    plt.title('Seasonal')
    plt.subplot(414)
    plt.plot(result.resid)
    plt.title('Residual')
    plt.tight_layout()
    plt.show()
    
    # 2. Autocorrelation check
    lag_acf = acf(y, nlags=365)
    plt.figure(figsize=(12, 6))
    plt.plot(lag_acf)
    plt.title('Autocorrelation Function')
    plt.axhline(y=0, linestyle='--', color='gray')
    plt.axhline(y=-1.96/np.sqrt(len(y)), linestyle='--', color='gray')
    plt.axhline(y=1.96/np.sqrt(len(y)), linestyle='--', color='gray')
    plt.tight_layout()
    plt.show()
    
    # 4. Statistical test for seasonality
    y_diff = y.diff().dropna()
    y_diff_square = y_diff ** 2
    result = stats.pearsonr(y_diff_square[:-12], y_diff_square[12:])
    print(f"Pearson correlation for seasonality: {result[0]:.4f}")
    print(f"p-value: {result[1]:.4f}")
    if result[1] < 0.05:
        print("The time series likely has seasonality (p < 0.05)")
    else:
        print("No strong evidence of seasonality (p >= 0.05)")

def check_stationarity(y):

    color_pal = sns.color_palette()
    plt.style.use('fivethirtyeight')
    
    result = adfuller(y)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:', result[4])
    if result[1] <= 0.05:
        print("The series is stationary")
    else:
        print("The series is not stationary")


X = preprocess_data(df)
display(X.head())

# Check for seasonality
check_seasonality(X['pm2.5'])
# Check for stationarity
check_stationarity(X['pm2.5'])

print("Latest datetime record (will be used for test set):", X.index.max())


X_ = X[X.index < '2014-12-28']

X_test_final = X[X.index >= '2014-12-28']

X = X_.copy()
display(X_test_final.head())
print(X.index.max())
print("Latest datetime record training set:", X.index.max())

## 3. Time Series Cross-Validation

Here, we implement and example of the time series cross-validation strategy we will follow: 

1. We use `TimeSeriesSplit` from scikit-learn to create a time series cross-validation strategy.
2. The data is split into 8 folds, with each test set being 60 days (with 24 hours each) in length.
3. We visualize each fold to understand how the data is being split over time.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

n = 8
tss = TimeSeriesSplit(n_splits=n, test_size=24*60*1, gap=0)
X = X.sort_index()

fig, axs = plt.subplots(n, 1, figsize=(25, 25), sharex=True)

fold = 0
for train_idx, val_idx in tss.split(X):
    train = X.iloc[train_idx]
    test = X.iloc[val_idx]
    train['pm2.5'].plot(ax=axs[fold],
                          label='Training Set',
                          title=f'Data Train/Test Split Fold {fold}')
    test['pm2.5'].plot(ax=axs[fold],
                         label='Test Set')
    axs[fold].axvline(test.index.min(), color='black', ls='--')
    fold += 1
plt.show()


## 4. Feature Engineering

In this section, we perform feature engineering:
1. The `create_features` function adds time-based features like hour, day of week, month, etc.
2. The `add_lags` function creates lag features (previous year's PM2.5 values) to capture yearly patterns.

In [ ]:
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    
    return df

def add_lags(df):
    target_map = df['pm2.5'].to_dict()
    df['lag1'] = (df.index - pd.Timedelta('91 days')).map(target_map)
    df['lag2'] = (df.index - pd.Timedelta('182 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('364 days')).map(target_map)
    return df


## 5. Model Training and Evaluation

Here, we define functions for model training, evaluation, and prediction:

1. `evaluate_model`: Computes various performance metrics (MSE, RMSE, MAE, R2) for a given model.
2. `train_model`: Implements the time series cross-validation strategy to train and evaluate multiple models.
3. `metrics_to_df`: Converts the performance metrics into a pandas DataFrame for easy visualization.
4. `future_predictions`: Uses a trained model to make predictions on the test set and visualize the results.
5. `create_folder_structure`: Creates folder structure for later store of models in pkl format and the plots

In [ ]:
def training(models, X, FEATURES, tss):
    metrics = {model: {'RMSE':[],'MAE':[],'MSE':[],'R2':[]} for model in models.keys()}

    preds = {'RMSE':[],'MAE':[],'MSE':[],'R2':[]}

    for train_idx, val_idx in tss.split(X):
        train = X.iloc[train_idx]
        test = X.iloc[val_idx]

        train = create_features(train)
        test = create_features(test) 
        
        model_store = {}
        for name, model in models.items():

            if name == "XGBoost":
                train = add_lags(train)
                test = add_lags(test)
                FEATURES_copy = FEATURES.copy()
                FEATURES += ['lag1','lag2','lag3']
            
            else:
                # remove lags for other models
                FEATURES = FEATURES_copy
            
            TARGET = 'pm2.5'

            X_train = train[FEATURES]
            y_train = train[TARGET]

            X_test = test[FEATURES]
            y_test = test[TARGET]

            preds = {'RMSE':[],'MAE':[],'MSE':[],'R2':[]}

            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            
            metrics_fold = evaluate_model(model,y_test,y_pred)

            for i in preds.keys():
                    metrics[name][i].append(metrics_fold[i])

            model_store[name] = model

    #Take the average of the metrics for each fold
    for model_name in models.keys():
        for i in preds.keys():
            metrics[model_name][i] = np.mean(metrics[model_name][i])

    return model_store, metrics

def metrics_to_df(metrics):
        #convert the metrics to a dataframe
    metrics_df = pd.DataFrame(metrics)
    metrics_df = metrics_df.transpose()
    metrics_df = metrics_df.reset_index()
    metrics_df = metrics_df.rename(columns={'index':'model'})
    #sort the dataframe by RMSE
    metrics_df = metrics_df.sort_values(by='RMSE')
    display(metrics_df)
    return metrics_df

def future_predictions(best_model_name,best_model,df,X_test_final):

    future_df = X_test_final
    future_df['isFuture'] = True
    df['isFuture'] = False
    df_and_future = pd.concat([df, future_df])
    df_and_future = create_features(df_and_future)
    if best_model_name == "XGBoost":
        df_and_future = add_lags(df_and_future)

    #display(X_test_final)

    FEATURES = ['DEWP', 'TEMP', 'PRES', 'Iws', 'hour', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth']

    if best_model_name == "XGBoost":
        FEATURES += ['lag1','lag2','lag3']

    print(f"Features used for prediction: {FEATURES}")
    future_w_features = df_and_future.query('isFuture').copy()
    #print(future_w_features.columns)

    # remove the target column
    future_w_features = future_w_features.drop(columns=['pm2.5','isFuture','dayofmonth'])

    #print(future_w_features.columns)

    
    future_w_features['pred'] = best_model.predict(future_w_features)

    # compute the metrics for the future predictions MAE, RMSE, MSE, R2
    metrics_future = evaluate_model(best_model,X_test_final['pm2.5'],future_w_features['pred'])
    pred_metrics = pd.DataFrame(metrics_future,index=[0])
    pred_metrics = pred_metrics.transpose()
    pred_metrics = pred_metrics.reset_index()
    pred_metrics = pred_metrics.rename(columns={'index':'metric',0:'value'})

    model_dir = os.path.join('models_results', best_model_name)
    os.makedirs(model_dir, exist_ok=True)

    X_test_final['pm2.5'].plot(figsize=(10, 5),
                                    ms=1,
                                    lw=1,
                                    colormap='viridis',
                                    legend=True,
                                    title='Predictions for model '+str(best_model_name) + " 100 steps ahead")

    future_w_features['pred'].plot(figsize=(10, 5),
                                    ms=1,
                                    lw=1,
                                    legend=True,
                                    title='Predictions for model '+str(best_model_name) + " 100 steps ahead")
    
    plt.savefig(os.path.join(model_dir, 'predictions.png'))
    plt.show()
    
def evaluate_model(model,y_test,y_pred):
    metrics = {}
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    metrics['MSE'] = mse
    metrics['RMSE'] = rmse
    metrics['MAE'] = mae
    metrics['R2'] = r2
    return metrics

def create_folder_structure():
    folders = [
        #'data',
        #'data',
        #'notebooks',
        #'src',
        'models_results',
        'models_pkl'
    ]
    for folder in folders:
        os.makedirs(folder, exist_ok=True)

## 7. Model Training and Evaluation - 

1. We reload and preprocess the data to ensure a fresh start.
2. The data is split into training and testing sets, with the test set containing data from December 28, 2014 onwards.
3. We create time-based features and lag features.
4. An grid of models  is defined with specific hyperparameters.
5. We use TimeSeriesSplit with 3 folds for cross-validation.
6. The model is trained and evaluated using our custom functions.
7. We print the performance metrics and generate predictions for the test set.

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
# for docker
df = pd.read_csv('beijing_pm_25.csv')
display(df.head())

# for local
#df = pd.read_csv('/Users/andresjr/Documents/capgemini/Task_2_new/data/beijing_pm_25.csv')
#display(df.head())


X = preprocess_data(df)

# The ending date for the test set is chosen as: 2014-12-28. The data before this date is used for training
X_ = X[X.index < '2014-12-28']

# The starting date for the test set is chosen as: 2014-12-28. The data after this date is used for testing
X_test_final = X[X.index >= '2014-12-28']

X = X_.copy()


# the test size is chosen as half year days and the gap between the training and test is chosen to 0. 
tss = TimeSeriesSplit(n_splits=3, test_size=12*365*1, gap=12)

models = {
        'XGBoost': xgb.XGBRegressor(base_score=0.5, booster='gbtree', 
                                    n_estimators=1500, 
                                    objective='reg:linear',
                                    max_depth=5,
                                    learning_rate=0.01),
        "randomforest": RandomForestRegressor(n_estimators=100, max_depth=3),
        "adaboost": AdaBoostRegressor(n_estimators=100),
    }

FEATURES = ['DEWP','TEMP', 'PRES', 'Iws','dayofyear', 'hour', 'dayofweek', 'quarter', 'month','year'] 


model_store, metrics = training(models, X, FEATURES, tss)

metrics_df = metrics_to_df(metrics)


for name, model in model_store.items():
    future_predictions(name,model,X,X_test_final)
    model_dir_pkl = os.path.join('models_pkl', name)
    os.makedirs(model_dir_pkl, exist_ok=True)
    joblib.dump(model, os.path.join(model_dir_pkl, 'model.pkl'))

metrics_df.to_csv('models_results/metrics.csv',index=False)

## Best model: XGBoost

The best model is XGBoost. This is likely because has ability to capture complex patterns in the time series data, effectively utilize the engineered features (including lag features), and regularization techniques that help prevent overfitting. 

As can be seen from the generated plot with the predictions, it manages to get an overview of the trend. I believe that with some of the potential improvements described in the next section (like hyperparameter tuning) or more complex models/features, a better estimate can be obtained.

In [ ]:
best_model_name = metrics_df.iloc[0]['model']
best_model = model_store[best_model_name]

print("This is the best model:", model_store)

future_predictions(best_model_name,best_model,X,X_test_final)

## 8. Conclusion

This notebook demonstrates a comprehensive approach to analyzing and predicting PM2.5 levels in Beijing. We've covered several key aspects of time series analysis and machine learning:

1. Data preprocessing and feature engineering
2. Checking for seasonality and stationarity in the time series
3. Implementing time series cross-validation
4. Training and evaluating multiple models, including XGBoost, Random Forest, and AdaBoost.
5. Visualizing model performance and predictions

The results provide insights into the effectiveness of different models in predicting PM2.5 levels, which can be valuable for air quality management and public health initiatives in Beijing.


## 9. Potential Improvements

To further enhance this analysis, consider the following improvements:

1. Hyperparameter tuning: Use techniques like GridSearchCV to optimize model parameters.
2. Feature importance analysis: Identify which features contribute most to the predictions.
3. Ensemble methods: Combine predictions from multiple models to potentially improve accuracy.
4. Deep learning approaches: Experiment with LSTM or other neural network architectures for time series prediction.
5. Use Prophet to get model prediction, a powerful library by Meta that yields much better predictions.
5. External data integration: Incorporate additional relevant data sources, such as traffic patterns or industrial activity.
6. Longer-term forecasting: Extend the prediction horizon to provide longer-term air quality forecasts.
7. Spatial analysis: If data is available, include spatial variations in PM2.5 levels across different areas of Beijing.

These enhancements could provide even more comprehensive and accurate predictions of PM2.5 levels, further supporting air quality management efforts in Beijing.